In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 12 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

09 January 2023 ; 17:53:26


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,17,0,8.8,1.5,3.1,0.500,...,1.21,112.47,111.20,1.27,1.11,112.45,111.24,1.21,1,1
11,MarJon Beauchamp,SF,22,MIL,28,8,15.6,2.3,5.5,0.413,...,1.21,112.47,111.20,1.27,1.11,112.45,111.24,1.21,1,1
17,A.J. Lawson,SG,22,DAL,5,0,1.8,0.2,0.8,0.250,...,1.41,116.45,115.03,1.42,1.61,116.51,114.90,1.61,1,1
25,Jaden Hardy,SG,20,DAL,14,0,10.1,2.1,4.9,0.420,...,1.41,116.45,115.03,1.42,1.61,116.51,114.90,1.61,1,1
36,AJ Griffin,SF,19,ATL,36,10,21.0,3.9,8.4,0.469,...,-1.15,113.29,114.42,-1.13,-1.44,113.35,114.75,-1.41,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,34.2,ORL,16.9,1.242424,3.975758,7.1,21.4,13,0.373791
1,Keegan Murray,28.8,SAC,10.7,0.227778,2.505556,4.2,11.6,5,0.128795
2,Bennedict Mathurin,28.0,IND,13.5,-0.800000,2.400000,5.3,17.1,6,0.120939
3,Jalen Duren,24.4,DET,15.7,0.210256,4.835897,3.3,7.7,15,0.087716
4,Walker Kessler,18.7,UTA,20.0,1.435000,6.150000,3.0,6.9,12,0.081824


In [7]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)